In [133]:
"""Import Python Modules"""
import os
import sys
import time
import numpy
import datetime

from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt

In [134]:
class LoginDetails():

    def __init__(self, username, password):
        self.username = username
        self.password = password
    
    ## Login details for Scihub
    def _sentinelAPI(self):
        return SentinelAPI(self.username, self.password, 'https://scihub.copernicus.eu/dhus')

In [135]:
class FootPrint():
    ## Read geojson file of watershed boundary
    def do_convertToGeojson(self, footprint):
        self.footprint = footprint
        return geojson_to_wkt(read_geojson(footprint))

In [136]:
class S2Downloader():
    def __init__(self, api):
        self.api = api
    
    ## Query Sentinel-2 Images using specified parameters 
    def do_queryProduct(self, footprint, startDate, endDate, processingLevel):
        self.footprint = footprint
        self.startDate = startDate
        self.endDate = endDate
        self.processinglevel = processingLevel
        
        return self.api.query(footprint, 
                        date = (startDate, endDate),
                        platformname = 'Sentinel-2',
                        processinglevel = processingLevel,
                        cloudcoverpercentage = (0, 100))
    
    ## Filter Sentinel-2 Images using tilename
    def do_filterProduct(self, product, tileName):
        self.product = product
        self.tileName = tileName
        
        s2tiles = [] 
        name = self.api.to_dataframe(product)['title']
        count = 0
        for tile in name:
            if tile.split("_")[5] == tileName:
                s2tiles.append(name.index[count])
            count += 1
        
        return s2tiles
    
    ## Download Sentinel-2 Images using specified parameters and tilename
    def do_download(self, tile, tileName, year, outDir):
        self.tile = tile
        self.outDir = outDir
        self.tileName = tileName
        self.year = year
        path = self.outDir + "\\" + "__" + self.year + "\\" + "__" + self.tileName 

        if not os.path.exists(path):
            os.makedirs(path)

        self.api.download_all(tile, path)

In [140]:
class S3Downloader():
    def __init__(self, api):
        self.api = api
    
    def do_queryProduct(self, footprint, startDate, endDate, productType):
        self.footprint = footprint
        self.startDate = startDate
        self.endDate = endDate
        self.productType = productType
        
        return self.api.query(footprint,
                        date = (startDate, endDate),
                        platformname = 'Sentinel-3',
                        producttype = productType)

In [151]:
def main():
    ## Watershed boundary using Geojson
    boundary = r'D:\__s2__\__boundary\watershedBoundary.geojson'
    
    ## Output Directory
    s2OutDir = r'D:\__s2__\__zip'
    s3OutDir = r'D:\__s3__\__zip'
    
    ## Username and Password for Copernicushub
    username = 'chriscandido93'
    password = 'nvenesyn5868'
    
    ## Query details
    startDate = '20200101'
    endDate = '20200131'

    ## Sentinel-2
    processingLevel = 'Level-2A'
    tileName = 'T51PUR'
    year = startDate[0:4]

    ## Sentinel-3
    productType = 'OL_1_EFR___'

    ## Login details
    login = LoginDetails(username, password)
    api = login._sentinelAPI()
    footprint = FootPrint().do_convertToGeojson(boundary)
    
    ## Sentinel-2 Downloader
    print ('\tStart Downloading ...')
    loopstarttime = str(datetime.datetime.now())
    print ("\t\tStart time: ", loopstarttime)
    startTime = time.time()
    s2Product = S2Downloader(api).do_queryProduct(footprint, startDate, endDate, processingLevel)
    s2FilterProduct = S2Downloader(api).do_filterProduct(s2Product, tileName)
    s2Download = S2Downloader(api).do_download(s2FilterProduct, tileName, year, s2OutDir)
    s2Download
    print ("\tDone.")       
    print("---\t\t %s seconds ---" % (time.time() - startTime))
    
    ## Sentinel-3 Downloader
    s3Product = S3Downloader(api).do_queryProduct(footprint, startDate, endDate, productType)

In [152]:
if __name__=="__main__":
    main()

c32ad201-291a-40e0-afe4-735f11db3c4e    S3B_OL_1_EFR____20200329T013234_20200329T01353...
81756d01-e15d-4514-a733-cd848c68fcfe    S3A_OL_1_EFR____20200329T021206_20200329T02150...
06351532-973b-4bbd-bfd9-56feb3ceb095    S3B_OL_1_EFR____20200328T015844_20200328T02014...
be9463ce-85ad-4caa-b8f5-7d008250cdf2    S3A_OL_1_EFR____20200326T014939_20200326T01523...
dbfdca91-2696-4c46-9ceb-9f5e3f9dab0a    S3A_OL_1_EFR____20200325T021550_20200325T02185...
                                                              ...                        
2d4c954f-80ea-4fc0-9685-9ca0f3b77d3f    S3A_OL_1_EFR____20200104T021547_20200104T02184...
eba5d250-bc25-4385-92f5-17611d4f3a84    S3B_OL_1_EFR____20200104T013616_20200104T01391...
48c866ab-e1cd-4569-b0c5-262e2ac85119    S3B_OL_1_EFR____20200103T020227_20200103T02052...
c82e137f-4abd-493c-ab2e-4f90f0139664    S3A_OL_1_EFR____20200102T012710_20200102T01301...
4ec8e27d-9019-4928-a31e-b3c9d2765795    S3A_OL_1_EFR____20200101T015321_20200101T01562...
Name: titl